In [ ]:
!pip install --ignore-installed sentencepiece langchain huggingface_hub

In [ ]:
!pip install git+https://github.com/huggingface/accelerate.git

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig, LlamaForCausalLM, BitsAndBytesConfig, pipeline, TextStreamer, StoppingCriteria, StoppingCriteriaList, logging
import textwrap
import locale
from langchain import HuggingFacePipeline
from transformers import pipeline
from langchain import PromptTemplate, LLMChain
locale.getpreferredencoding = lambda: "UTF-8"
import torch
import transformers
import torch.nn as nn

In [4]:
model_id = "/workspace/axolotl/src/out"

In [5]:
# Reload model in FP16 and merge it with LoRA weights
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype = torch.bfloat16,
)

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
print("Model & tokenizer loaded")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model & tokenizer loaded


In [6]:
streamer = TextStreamer(tokenizer, skip_prompt=True)
stream_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    temperature=0.6,
    top_p=0.9,
    do_sample=True,
    top_k=50,
    repetition_penalty=1.15,
    batch_size=8,
    streamer=streamer,
)
stream_llm = HuggingFacePipeline(pipeline=stream_pipeline)
stream_template = f'''{tokenizer.bos_token}<|system|>Act as an helpful assistant. Whenever a user poses a question or presents an issue, respond with clear, accurate, and comprehensive solutions or explanations tailored to address the query. Prioritize user clarity and the relevancy of your response.{tokenizer.eos_token}
<|user|>user\n'''
stream_template += "{query}"
stream_template += f'''{tokenizer.eos_token}<|assistant|>\n'''

stream_prompt = PromptTemplate(template=stream_template, input_variables=["query"])
stream_llm_chain = LLMChain(prompt=stream_prompt, llm=stream_llm)

In [13]:
stream_llm_chain.run('''Write code fetch google news api using nodejs.''')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


To fetch data from the Google News API using Node.js, we will use the `request` library to make HTTP requests and `cheerio` to parse HTML content. If you don't have these libraries installed, you can install them using npm (Node Package Manager) by running:

```bash
npm install request cheerio
```

Here is the complete code solution for fetching Google News articles:

```javascript
// Import necessary modules
var request = require('request'); // To make HTTP requests
var cheerio = require('cheerio'); // To parse HTML content

// Function to fetch news from Google News API
function fetchGoogleNews(callback) {
   // The URL for the Google News API
   var url = 'https://news.google.com/rss';
   
   // Request options
   var options = {
       url: url,
       headers: {
           'User-Agent': 'Request-Headers' // Set the User-Agent to avoid being blocked
       }
   };
   
   // Make an HTTP GET request to the Google News API
   request(options, function(error, response, body) {
       

'To fetch data from the Google News API using Node.js, we will use the `request` library to make HTTP requests and `cheerio` to parse HTML content. If you don\'t have these libraries installed, you can install them using npm (Node Package Manager) by running:\n\n```bash\nnpm install request cheerio\n```\n\nHere is the complete code solution for fetching Google News articles:\n\n```javascript\n// Import necessary modules\nvar request = require(\'request\'); // To make HTTP requests\nvar cheerio = require(\'cheerio\'); // To parse HTML content\n\n// Function to fetch news from Google News API\nfunction fetchGoogleNews(callback) {\n    // The URL for the Google News API\n    var url = \'https://news.google.com/rss\';\n    \n    // Request options\n    var options = {\n        url: url,\n        headers: {\n            \'User-Agent\': \'Request-Headers\' // Set the User-Agent to avoid being blocked\n        }\n    };\n    \n    // Make an HTTP GET request to the Google News API\n    reques